<a href="https://colab.research.google.com/github/italolima04/monitoria-probabilidade-estatistica/blob/master/Implementa%C3%A7%C3%A3o-Pesquisa-PID/Modelos-N%C3%A3o-Supervisionados/K_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importando algoritmo KMeans da biblioteca Sklearn.
from sklearn.cluster import KMeans

In [ ]:
#Carregando modelo Word2VEC.
word_vectors = w2v_model.wv
#Criando instância do algoritmo K-Means e passando como entrada os vetores originados pelo Word2VEC.
#n_clusters=2, max_iter=1000, random_state=True, n_init=50
model = KMeans(n_clusters=2, init='k-means++', max_iter=2000, random_state=True, n_init=50).fit(X=word_vectors.vectors)

In [ ]:
negative_cluster_center = model.cluster_centers_[1]
positive_cluster_center = model.cluster_centers_[0]

In [ ]:
#Verificando os vetores que foram entradas para o algoritmo, criados pelo modelo Word2VEC.
word_vectors.vectors

In [ ]:
model.cluster_centers_

In [ ]:
#Calculando a distância de cada ponto de dado para o centroide correspondente.
distance = model.fit_transform(X=word_vectors.vectors)

In [ ]:
distance

In [ ]:
len(model.labels_)

In [ ]:
#Verificando os labels originados pelo modelo.
labels = model.labels_
labels

In [ ]:
#Criando DataFrame com as classificações para os tweets.
df_kmeans = pd.DataFrame(data=model.labels_, columns=['Cluster'])

In [ ]:
#Visualizando o novo DataFrame criado.
df_kmeans

In [ ]:
#Distruibuição das sentenças nos clusters pelo algoritmo K-Means.
df_kmeans['Cluster'].value_counts()

In [ ]:
#Porcentagem que representa a distribuição das sentenças nos clusters pelo algoritmo K-Means.
df_kmeans['Cluster'].value_counts(normalize=True)

In [ ]:
#Configurando os parâmetros da Figura.
plt.figure(figsize=(16, 8))


#Plotando Gráfico com Matplotlib.
df_kmeans['Cluster'].value_counts().plot(kind='bar', color=['blue', 'red'],  alpha=0.8, width=0.2)


#Plotando Gráfico com Searborn.

#Configurações para as fontes dos textos.
font = {'family': 'serif',
        'weight': 'bold',
        'size': 16,
        }
plt.title('Gráfico para distribuição de Classes - Clusters', fontdict=font, color='white', size=30)

#Configurando título, labels e ticks
plt.xlabel('Meses de Coleta', fontdict=font, color='white', size=30)
plt.ylabel('Tweets Coletados', fontdict=font, color='white', size=30)

#Configuração de exibição dos ticks.
plt.xticks(rotation=45, size=25, color='black', weight="bold")
plt.yticks(size=25, color='black', weight="bold")

#Exibindo a figura.
plt.savefig('grafico.png', format='png', quality=100)
plt.show()

In [ ]:
#Verificando os clusters únicos.
df_kmeans['Cluster'].unique()

In [ ]:
#Criando Vocabulário em formato de Dicionário para o sentimento das palavras.
words = pd.DataFrame(word_vectors.vocab.keys())
#Nomeando a coluna do DataFrame.
words.columns = ['words']
#Atribuindo as palavras a classe positiva ou negativa, de acordo com a clusterização.
words['vectors'] = words.words.apply(lambda x: word_vectors.wv[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [ ]:
#Para o cluster 0 atribuiu-se o sentimento como positivo e para o outro cluster atribuiu-se o sentimento negativo.
words['cluster_value'] = [1 if i==0 else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [ ]:
#Visualizando as 5 primeiras linhas.
words.head(5)

In [ ]:
#Filtrando os dados atribuídos como negativo.
negativas = words.loc[words.sentiment_coeff < 0, ['words', 'cluster', 'sentiment_coeff']]
negativas

In [ ]:
#Filtrando os dados atribuídos como positivo.
positivas = words.loc[words.sentiment_coeff > 0, ['words', 'cluster', 'sentiment_coeff']]
positivas

In [ ]:
#Verificando a estrutura do DataFrame.
words.shape

In [ ]:
#Descrevendo estatisticamente os valores associados ao Coeficiente de Sentimento.
words.describe()['sentiment_coeff']

In [ ]:
#Importando Módulo da biblioteca Plotly,
import plotly.graph_objects as go

#Instanciando uma figura.
fig = go.Figure()

#Configurando e criando a exibição do gráfico.
fig.add_trace(go.Box(y=words['sentiment_coeff'], name='Score de Sentimento', marker_color = 'black'))

#Configurando o Layout a ser exibido.
fig.update_layout(width=700, height=700)

#Exibindo imagem.
fig.show()

In [ ]:
#Exportando o dicionário de sentimentos criados, no formato CSV.
words[['words', 'sentiment_coeff']].to_csv('dicionario_sentimentos.csv', index=False)

**Implementando Métricas e Visualizando Graficamente os índices.**

In [ ]:
from sklearn import metrics
from sklearn.metrics import silhouette_score 
from sklearn.metrics import davies_bouldin_score

In [ ]:
silhouette_score(word_vectors.vectors, model.labels_)

In [ ]:
metrics.calinski_harabasz_score(word_vectors.vectors, labels)

In [ ]:
davies_bouldin_score(word_vectors.vectors, labels)

In [ ]:
word_vectors = w2v_model.wv

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

model = KMeans(2, random_state=42)
visualizer = SilhouetteVisualizer(model, colors='yellowbrick')
visualizer.fit(word_vectors.vectors)        

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,11), metric='silhouette', timings=False)
visualizer.fit(word_vectors.vectors);  

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,11), metric='calinski_harabaz', timings=False)
visualizer.fit(word_vectors.vectors);        

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

from yellowbrick.text import TSNEVisualizer

tfidf = TfidfVectorizer()

X = tfidf.fit_transform(words)
y = words['cluster']

tsne = TSNEVisualizer()
tsne.fit(X, y)
tsne.show()